# Train a Cornac Model using Script Mode

## Setup Sagemaker

In [1]:
# %pip install -U sagemaker

In [2]:
# %pip install sagemaker-experiments

In [1]:
import boto3
import pandas as pd
import numpy as np
import time
import sagemaker
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup

In [2]:
%env AWS_PROFILE=shuxian-sagemaker

env: AWS_PROFILE=shuxian-sagemaker


In [3]:
!aws sts get-caller-identity

{
    "UserId": "AROAWC4YSIQL5OBFCNGEX:botocore-session-1687241421",
    "Account": "418542404631",
    "Arn": "arn:aws:sts::418542404631:assumed-role/SageMaker-UserRole/botocore-session-1687241421"
}


In [4]:
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='SageMaker-UserRole')['Role']['Arn']

region = boto3.Session().region_name
print(f'Current region: {region}')

boto_session = boto3.Session(region_name=region)
sagemaker_session = sagemaker.Session(boto_session=boto_session)
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
sagemaker_client.list_feature_groups()

featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

Couldn't call 'get_role' to get Role ARN from role name SageMaker-UserRole to get Role path.


Current region: ap-southeast-1


In [5]:
sagemaker_client.list_feature_groups()

{'FeatureGroupSummaries': [{'FeatureGroupName': 'users-feature-group',
   'FeatureGroupArn': 'arn:aws:sagemaker:ap-southeast-1:418542404631:feature-group/users-feature-group',
   'CreationTime': datetime.datetime(2023, 6, 19, 18, 52, 18, 285000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'interactions-feature-group',
   'FeatureGroupArn': 'arn:aws:sagemaker:ap-southeast-1:418542404631:feature-group/interactions-feature-group',
   'CreationTime': datetime.datetime(2023, 6, 19, 18, 55, 22, 260000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'cats-feature-group',
   'FeatureGroupArn': 'arn:aws:sagemaker:ap-southeast-1:418542404631:feature-group/cats-feature-group',
   'CreationTime': datetime.datetime(2023, 6, 19, 18, 53, 28, 829000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Ac

In [6]:
# users = pd.DataFrame()
users_feature_group = FeatureGroup(name="users-feature-group", sagemaker_session=sagemaker_session)

# get single record from user
record_identifier_value = str("079b0ec9-cec6-42fb-9f00-7891c52a10fb")
featurestore_runtime.get_record(FeatureGroupName="users-feature-group",
                                RecordIdentifierValueAsString=record_identifier_value)

{'ResponseMetadata': {'RequestId': 'ddd14183-50af-4561-9c39-aed0ce1f6b6c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ddd14183-50af-4561-9c39-aed0ce1f6b6c',
   'content-type': 'application/json',
   'content-length': '1661',
   'date': 'Tue, 20 Jun 2023 06:12:10 GMT'},
  'RetryAttempts': 0},
 'Record': [{'FeatureName': 'id',
   'ValueAsString': '079b0ec9-cec6-42fb-9f00-7891c52a10fb'},
  {'FeatureName': 'has_other_cats', 'ValueAsString': '0'},
  {'FeatureName': 'personality', 'ValueAsString': 'all sweet'},
  {'FeatureName': 'gender', 'ValueAsString': 'no preference'},
  {'FeatureName': 'good_with_other_dogs', 'ValueAsString': '0'},
  {'FeatureName': 'employment', 'ValueAsString': 'working full time'},
  {'FeatureName': 'created_at', 'ValueAsString': '2023-05-18T17:13:45Z'},
  {'FeatureName': 'agree_to_fee', 'ValueAsString': '1'},
  {'FeatureName': 'is_first_cat', 'ValueAsString': '1'},
  {'FeatureName': 'good_with_kids', 'ValueAsString': '0'},
  {'FeatureName': 'att

In [7]:
# cat images
from sagemaker.feature_store.feature_store import FeatureStore

sagemaker_session = sagemaker.Session()
data_name = 'cat-images'
feature_group_name = f'{data_name}-feature-group'
feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=sagemaker_session)

feature_store = FeatureStore(feature_store_session)
builder = feature_store.create_dataset(
    base=feature_group,
    output_path=f's3://petfinder6000-auxiliary/{data_name}'
)

df_images, query = builder.to_dataframe()

In [8]:
df_images.head()

,id,cws_id,updated_at,img_shape,feature_vectors
0,b890540f-6dae-43e0-99b3-db93d971c17d,1681491763-346,2023-05-15T02:51:44Z,"[128, 128, 3]","[0.0, 2.2147867679595947, 0.0, 0.0, 0.0, 0.0, ..."
1,c8ea7e3c-4471-4e7a-bb57-62de65de6145,1681491514-236,2023-05-15T02:51:13Z,"[128, 128, 3]","[0.0, 1.9950363636016846, 0.0, 0.0, 0.0, 0.0, ..."
2,dfb27ef3-57fe-4c6a-98b6-a37e6ba8037a,1681491779-353,2023-05-15T02:51:45Z,"[128, 128, 3]","[0.0, 2.0274159908294678, 0.0, 0.0, 0.0, 0.0, ..."
3,ccd28ed0-2563-42ec-8d38-62104c1476e8,1681491080-42,2023-05-15T02:51:22Z,"[128, 128, 3]","[0.0, 2.025311231613159, 0.0, 0.0, 0.0, 0.0, 0..."
4,161d398a-3b51-47e4-849c-a8b1fb181db8,1681491643-294,2023-05-15T02:51:38Z,"[128, 128, 3]","[0.0, 1.9953548908233643, 0.0, 0.0, 0.0, 0.0, ..."


In [9]:
# extract catID and feature vectors

catID = df_images['id']

In [10]:
# function to remap vectors from a list to array
def list_to_arr(str_list):
    # split the stringed list
    arr = [float(num.strip(',')) for num in str_list[1:-1].split()]
    arr = np.array(arr)
    return arr


### Feature vectors
# Arrayify
img_features=df_images['feature_vectors'].map(list_to_arr)

# Reshape
array_data = np.array([np.array(x) for x in img_features])
img_features = array_data.reshape((array_data.shape[0], -1))

img_features.shape

(404, 20480)

In [11]:
df_image_features = pd.DataFrame(img_features)
df_image_features

,0,1,2,3,4,5,6,7,8,9,...,20470,20471,20472,20473,20474,20475,20476,20477,20478,20479
0,0.0,2.214787,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.235870,0.0,0.0,0.0,0.530021,0.0,0.0,0.0,0.0,0.0
1,0.0,1.995036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.386383,0.0,0.0,0.0,0.701624,0.0,0.0,0.0,0.0,0.0
2,0.0,2.027416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.556160,0.0,0.0,0.0,0.681977,0.0,0.0,0.0,0.0,0.0
3,0.0,2.025311,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.395177,0.0,0.0,0.0,0.514185,0.0,0.0,0.0,0.0,0.0
4,0.0,1.995355,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.619329,0.0,0.0,0.0,0.591742,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,0.0,2.040782,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.556398,0.0,0.0,0.0,0.659501,0.0,0.0,0.0,0.0,0.0
400,0.0,2.038180,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.577067,0.0,0.0,0.0,0.537965,0.0,0.0,0.0,0.0,0.0
401,0.0,1.924961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.468211,0.0,0.0,0.0,0.634498,0.0,0.0,0.0,0.0,0.0
402,0.0,1.969521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.557976,0.0,0.0,0.0,0.706427,0.0,0.0,0.0,0.0,0.0


In [12]:
# catID to s3
catID.to_csv("catID.csv", header=False, index=False)
s3_catID = sagemaker_session.upload_data(path="catID.csv", key_prefix="petfinder6000-auxiliary")
print(s3_catID)


s3://sagemaker-ap-southeast-1-418542404631/petfinder6000-auxiliary/catID.csv


In [13]:
# image features to s3
df_image_features.to_csv("image_features.csv", header=False, index=False)
s3_image_features = sagemaker_session.upload_data(path="image_features.csv", key_prefix="petfinder6000-auxiliary")
print(s3_image_features)

s3://sagemaker-ap-southeast-1-418542404631/petfinder6000-auxiliary/image_features.csv


## Train model
The model is trained using the SageMaker SDK's Estimator class. Firstly, get the execution role for training. This role allows us to access the S3 bucket in the last step, where the train and test data set is located.

In [14]:
# TODO: add code to retrieve latest train/test split based on type of split
split_type = "strat"
object_name = "strat-2023-06-16-07-09-28-233"
train_uri = f"s3://petfinder6000-training/{object_name}/output/train/train.csv"
eval_uri = f"s3://petfinder6000-training/{object_name}/output/validation/validation.csv"
test_uri = f"s3://petfinder6000-training/{object_name}/output/test/test.csv"
image_features_uri = f"s3://sagemaker-ap-southeast-1-418542404631/petfinder6000-auxiliary/image_features.csv"
catID_uri = f"s3://sagemaker-ap-southeast-1-418542404631/petfinder6000-auxiliary/catID.csv"

In [15]:
def get_uri_instance(run_mode):
    if run_mode == 'LOCAL':
        image_uri = 'cornac-39' # can pull remote container from ECR too
        instance_type = 'local'
    else:
        image_uri = '418542404631.dkr.ecr.ap-southeast-1.amazonaws.com/petfinder6000:cornac-39-v1'
        instance_type = 'ml.m5.xlarge'
    return image_uri, instance_type

In [18]:
# Docs: https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html
# https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker-experiments/sagemaker_job_tracking/tensorflow_script_mode_training_job.ipynb

from sagemaker.tensorflow import TensorFlow
from sagemaker.inputs import TrainingInput
from sagemaker.experiments.run import Run

exp_name = 'vbpr'
run_name = 'vbpr-' + time.strftime("%Y%m%d-%H-%M-%S", time.gmtime())
job_prefix = f"petfinder6000/training"

with Run(experiment_name=exp_name, run_name=run_name, sagemaker_session=sagemaker_session) as run:
    # setup hyperparameters
    k=10               # dimension of the gamma latent factors
    k2=10              # dimension of the tetha latent factors
    n_epochs=20
    batch_size=100
    learning_rate=0.005
    lambda_w=0.01      # regularization hyperparameter for latent factor weights
    lambda_b=0.01      # regularization hyperparameter for biases
    lambda_e=0.0       # regularization hyperparameter for embedding matrix E and beta prime vector

    run.log_parameter("k", k)
    run.log_parameter("k2", k2)
    run.log_parameter("n_epochs", n_epochs)
    run.log_parameter("batch_size", batch_size)
    run.log_parameter("learning_rate", learning_rate)
    run.log_parameter("lambda_w", lambda_w)
    run.log_parameter("lambda_b", lambda_b)
    run.log_parameter("lambda_e", lambda_e)


    run_mode = 'LOCAL'
    image_uri, instance_type = get_uri_instance(run_mode)

    estimator = TensorFlow(
        image_uri=image_uri,
        entry_point="train_vbpr.py", # Define the model file here
        source_dir="./training",
        dependencies=[
            './training/metrics/harmonic_mean.py',
            './training/metrics/combined_eval_method.py',
            './training/metrics/serendipity_wrapper.py'
        ],
        role=role,
        instance_count=1,
        instance_type=instance_type,
        base_job_name=job_prefix,
        environment={"REGION": region, "EXP_NAME": exp_name, "RUN_NAME": run_name},
        hyperparameters={
            "k": k,
            "k2": k2,
            "n_epochs": n_epochs,
            "batch_size": batch_size,
            "learning_rate": learning_rate,
            "lambda_w": lambda_w,
            "lambda_b": lambda_b,
            "lambda_e": lambda_e,
        },
    )

    # train estimator
    estimator.fit(
        inputs={
            "train": TrainingInput(s3_data=train_uri, content_type="text/csv"),
            "eval": TrainingInput(s3_data=eval_uri, content_type="text/csv"),
            "image_features": TrainingInput(s3_data=image_features_uri, content_type="text/csv"),
            "catID": TrainingInput(s3_data=catID_uri, content_type="text/csv")
        },
        job_name=f"{job_prefix}/{run_name}",
        wait=False
    )

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:sagemaker:Creating training-job with name: petfinder6000/training/vbpr-20230620-06-45-37
INFO:sagemaker.local.local_session:Starting training job
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-rdk5l:
    command: train
    container_name: hhaykkmahu-algo-1-rdk5l
    environment:
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    image: cornac-39
    networks:
      sagemaker-local:
        aliases:
        - algo-1-rdk5l
    stdin_open: true
    tty: true
    volumes:
    - /private/var/folders/0g/_vw0ydz94_x5cpswktv47t9h0000gn/T/tmpapqffuuf/algo-1-rdk5l/output/data:/opt/ml/output/data
    - /private/var/folders/0g/_vw0ydz94_x5cpswktv47t9h0000gn/T/tmpapqffuuf/algo-1-rdk5l/input:/opt/ml/input
    - /private/var/folders/0g/_vw0ydz9

Network sagemaker-local  Creating
Network sagemaker-local  Created
Container hhaykkmahu-algo-1-rdk5l  Creating
Container hhaykkmahu-algo-1-rdk5l  Created
Attaching to hhaykkmahu-algo-1-rdk5l
hhaykkmahu-algo-1-rdk5l  | 2023-06-20 06:45:47,138 botocore.credentials INFO     Found credentials in environment variables.
hhaykkmahu-algo-1-rdk5l  | 2023-06-20 06:45:47,421 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
hhaykkmahu-algo-1-rdk5l  | /usr/local/bin/python -m pip install -r requirements.txt
hhaykkmahu-algo-1-rdk5l  | Requirement already satisfied: sagemaker-training==4.5.0 in /usr/local/lib/python3.9/site-packages (from -r requirements.txt (line 1)) (4.5.0)
hhaykkmahu-algo-1-rdk5l  | Collecting sagemaker-experiments==0.1.45
hhaykkmahu-algo-1-rdk5l  |   Downloading sagemaker_experiments-0.1.45-py3-none-any.whl (42 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 3.8 MB/s eta 0:00:00 eta -:--:--
hhaykkmahu-algo-1-rdk5l  | Collecting pan

INFO:root:copying /private/var/folders/0g/_vw0ydz94_x5cpswktv47t9h0000gn/T/tmpapqffuuf/algo-1-rdk5l/output/failure -> /private/var/folders/0g/_vw0ydz94_x5cpswktv47t9h0000gn/T/tmpapqffuuf/artifacts/output
INFO:root:creating /private/var/folders/0g/_vw0ydz94_x5cpswktv47t9h0000gn/T/tmpapqffuuf/artifacts/output/data


hhaykkmahu-algo-1-rdk5l exited with code 1
Aborting on container exit...
Container hhaykkmahu-algo-1-rdk5l  Stopping
Container hhaykkmahu-algo-1-rdk5l  Stopped


RuntimeError: Failed to run: ['docker-compose', '-f', '/private/var/folders/0g/_vw0ydz94_x5cpswktv47t9h0000gn/T/tmpapqffuuf/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1

### Evaluate Model

In [9]:
from sagemaker.workflow.properties import PropertyFile
from sagemaker.processing import FrameworkProcessor, ProcessingInput, ProcessingOutput
from sagemaker.sklearn import SKLearn

evaluation_prefix = f"{job_prefix}/evaluation"
report_path = f"{evaluation_prefix}/report"
model_path = estimator.model_data.replace("\\","/")
print(f"Model is stored in: {model_path}")

evaluate_model_processor = FrameworkProcessor(
    role=role,
    image_uri=image_uri,
    base_job_name=evaluation_prefix,
    estimator_cls=SKLearn,
    framework_version='0.23-1',
    command=["python3"],
    instance_count=1,
    instance_type=instance_type,
    sagemaker_session=sagemaker_session,
)

evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json"
)

eval_args = evaluate_model_processor.run(
    inputs=[
        ProcessingInput(source=model_path, destination="/opt/ml/processing/model"),
        ProcessingInput(source=train_uri, destination="/opt/ml/processing/train"),
        ProcessingInput(source=test_uri, destination="/opt/ml/processing/test"),
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation", destination=report_path),
    ],
    code="evaluate.py",
    source_dir="./evaluation",
    dependencies=[
        './evaluation/metrics/harmonic_mean.py',
        './evaluation/metrics/combined_eval_method.py',
        './evaluation/metrics/serendipity_wrapper.py'
    ],
    job_name=evaluation_prefix,
)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


Model is stored in: s3://sagemaker-ap-southeast-1-418542404631/petfinder6000/training/bpr-20230619-01-05-27/model.tar.gz


INFO:sagemaker.processing:Uploaded ./evaluation to s3://sagemaker-ap-southeast-1-418542404631/petfinder6000/training/evaluation/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ap-southeast-1-418542404631/petfinder6000/training/evaluation/source/runproc.sh
INFO:sagemaker:Creating processing-job with name petfinder6000/training/evaluation
INFO:sagemaker.local.local_session:Starting processing job
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-03agc:
    container_name: 5qxxwvtlig-algo-1-03agc
    entrypoint:
    - /bin/bash
    - /opt/ml/processing/input/entrypoint/runproc.sh
    environment:
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    image: cornac-39
    networks:
      sagemaker-local:
        aliases:
        - algo-1-03agc
    stdin_open: true
    tty: true
    volumes:
    - C:\Users\yongr\AppData\Local\Temp\tmp9xn7j7xb\algo-1-03agc\output:/opt/ml/output
  

Container 5qxxwvtlig-algo-1-03agc  Creating
Container 5qxxwvtlig-algo-1-03agc  Created
Attaching to 5qxxwvtlig-algo-1-03agc
5qxxwvtlig-algo-1-03agc  | WARNING: Skipping typing as it is not installed.
5qxxwvtlig-algo-1-03agc  | WARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv
5qxxwvtlig-algo-1-03agc  | Collecting pandas>=1.5.0
5qxxwvtlig-algo-1-03agc  |   Downloading pandas-2.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 11.9 MB/s eta 0:00:0000:0100:010m
5qxxwvtlig-algo-1-03agc  | Collecting recommenders>=1.1.0
5qxxwvtlig-algo-1-03agc  |   Downloading recommenders-1.1.1-py3-none-any.whl (339 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.0/339.0 kB 9.2 MB/s eta 0:00:0000:01m-:--:--
5qxxwvtlig-algo-1-03agc  | Collecting pytz>=202

### Register Trained Model in Model Registry

In [15]:
from sagemaker.model import Model
from sagemaker.model_metrics import MetricsSource, ModelMetrics

inference_instance_type = "ml.m5.xlarge"
ecr_image = '418542404631.dkr.ecr.ap-southeast-1.amazonaws.com/petfinder6000:cornac-39-v1'

print(f"Location of model: {model_path}")
model = Model(
    image_uri=ecr_image,
    model_data=model_path,
    role=role,
    sagemaker_session=sagemaker_session
)

evaluation_s3_uri = f"s3://{report_path}/output/evaluation/evaluation.json"

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=evaluation_s3_uri,
        content_type="application/json",
    )
)

model_package_group_name = "TestModelPackageGroup"
model_package = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=[inference_instance_type],
    transform_instances=[inference_instance_type],
    model_metrics=model_metrics,
    model_package_group_name=model_package_group_name,
    approval_status="PendingManualApproval",
)

model_package_arn = model_package.model_package_arn
print("Model Package ARN : ", model_package_arn)

Location of model: s3://sagemaker-ap-southeast-1-418542404631/petfinder6000/training/bpr-20230619-01-05-27/model.tar.gz
Model Package ARN :  arn:aws:sagemaker:ap-southeast-1:418542404631:model-package/testmodelpackagegroup/1


## Cleanup

### Delete experiments

In [45]:
from sagemaker.experiments.experiment import Experiment

experiment_name = "tensorflow-script-mode-experiment"
exp = Experiment.load(experiment_name=experiment_name, sagemaker_session=sagemaker_session)
print(exp)
exp._delete_all(action="--force")

Experiment(sagemaker_session=<sagemaker.session.Session object at 0x0000026ECD0BC670>,experiment_name='tensorflow-script-mode-experiment',experiment_arn='arn:aws:sagemaker:ap-southeast-1:418542404631:experiment/tensorflow-script-mode-experiment',display_name='tensorflow-script-mode-experiment',creation_time=datetime.datetime(2023, 6, 9, 22, 44, 58, 124000, tzinfo=tzlocal()),created_by={'UserProfileArn': 'arn:aws:sagemaker:ap-southeast-1:418542404631:user-profile/d-ni9jmwq8akiv/aeroxye', 'UserProfileName': 'aeroxye', 'DomainId': 'd-ni9jmwq8akiv'},last_modified_time=datetime.datetime(2023, 6, 10, 16, 30, 12, 866000, tzinfo=tzlocal()),last_modified_by={},response_metadata={'RequestId': 'd6609acb-9f9e-4700-97f8-235f2b7ffa68', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd6609acb-9f9e-4700-97f8-235f2b7ffa68', 'content-type': 'application/x-amz-json-1.1', 'content-length': '472', 'date': 'Wed, 14 Jun 2023 08:22:31 GMT'}, 'RetryAttempts': 0})
